# Algonauts Challenge

Modified algonauts' challenge template for training VAE-based model

## Environment configuraton

### User parameters

In [ ]:
from traitlets.traitlets import ForwardDeclaredInstance

rand_seed = 5 #@param {allow-input: true}

platform = 'colab' #@param ['colab', 'jupyter_notebook'] {allow-input: true}

device = 'cuda' #@param ['cpu', 'cuda'] {allow-input: true}

subj = 6 #@param ["1", "2", "3", "4", "5", "6", "7", "8"] {type:"raw", allow-input: true}

batch_size = 200 #@param {type:"integer", allow-input: true}

latent_dim = 100 #@param {type:"integer", allow-input: true}

regressor = 'linear' #@param ['linear', 'mlp'] {allow-input: true}

vae_model_path = '/NeuroAI/vae_100' #@param {type:"string", allow-input: true}

regressor_model_path = '/NeuroAI/linear_s6' #@param {type:"string", allow-input: true}

write_out_regressor = False #@param {type:"boolean", allow-input: true}

### Set up environment

In [ ]:
!git clone https://github.com/AntixK/PyTorch-VAE

fatal: destination path 'PyTorch-VAE' already exists and is not an empty directory.


In [ ]:
!cd PyTorch-VAE

In [ ]:
!pip install -r PyTorch-VAE/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
sys.path.insert(0, '/content/PyTorch-VAE')

In [ ]:
import os
import numpy as np
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import matplotlib
from matplotlib import pyplot as plt

import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms
from scipy.stats import pearsonr as corr
from torchsummary import summary

from models import VanillaVAE

In [ ]:
if platform == 'colab':
    from google.colab import drive
    drive.mount('/content/drive/', force_remount=True)
    data_dir = '/content/drive/MyDrive/algonauts_2023_tutorial_data' #@param {type:"string"}
    parent_submission_dir = '/content/drive/MyDrive/algonauts_2023_challenge_submission' #@param {type:"string"}
elif platform == 'jupyter_notebook':
    data_dir = '../algonauts_2023_challenge_data'
    parent_submission_dir = '../algonauts_2023_challenge_submission'

Mounted at /content/drive/


In [ ]:
device = 'cuda' #@param ['cpu', 'cuda'] {allow-input: true}
device = torch.device(device)

In [ ]:
class argObj:
  def __init__(self, data_dir, parent_submission_dir, subj):
    
    self.subj = format(subj, '02')
    self.data_dir = os.path.join(data_dir, 'subj'+self.subj)
    self.parent_submission_dir = parent_submission_dir
    self.subject_submission_dir = os.path.join(self.parent_submission_dir,
        'subj'+self.subj)

    # Create the submission directory if not existing
    if not os.path.isdir(self.subject_submission_dir):
        os.makedirs(self.subject_submission_dir)

args = argObj(data_dir, parent_submission_dir, subj)

## Preprocess data

### Load voxel data

In [ ]:
fmri_dir = os.path.join(args.data_dir, 'training_split', 'training_fmri')
lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))

print('LH training fMRI data shape:')
print(lh_fmri.shape)
print('(Training stimulus images × LH vertices)')

print('\nRH training fMRI data shape:')
print(rh_fmri.shape)
print('(Training stimulus images × RH vertices)')

LH training fMRI data shape:
(9082, 18978)
(Training stimulus images × LH vertices)

RH training fMRI data shape:
(9082, 20220)
(Training stimulus images × RH vertices)


### Load images

In [ ]:
train_img_dir  = os.path.join(args.data_dir, 'training_split', 'training_images')
test_img_dir  = os.path.join(args.data_dir, 'test_split', 'test_images')

# Create lists will all training and test image file names, sorted
train_img_list = os.listdir(train_img_dir)
train_img_list.sort()
test_img_list = os.listdir(test_img_dir)
test_img_list.sort()
print('Training images: ' + str(len(train_img_list)))
print('Test images: ' + str(len(test_img_list)))

Training images: 9082
Test images: 293


In [ ]:
train_img_file = train_img_list[0]
print('Training image file name: ' + train_img_file)
print('73k NSD images ID: ' + train_img_file[-9:-4])

Training image file name: train-0001_nsd-00004.png
73k NSD images ID: 00004


### Spit into train/test

In [ ]:
np.random.seed(rand_seed)

# Calculate how many stimulus images correspond to 90% of the training data
num_train = int(np.round(len(train_img_list) / 100 * 90))
# Shuffle all training stimulus images
idxs = np.arange(len(train_img_list))
np.random.shuffle(idxs)
# Assign 90% of the shuffled stimulus images to the training partition,
# and 10% to the test partition
idxs_train, idxs_val = idxs[:num_train], idxs[num_train:]
# No need to shuffle or split the test stimulus images
idxs_test = np.arange(len(test_img_list))

print('Training stimulus images: ' + format(len(idxs_train)))
print('\nValidation stimulus images: ' + format(len(idxs_val)))
print('\nTest stimulus images: ' + format(len(idxs_test)))

Training stimulus images: 8174

Validation stimulus images: 908

Test stimulus images: 293


### Create data pipeline

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), # resize the images to 224x224 pixels
    transforms.ToTensor(), # convert the images to a PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalize the images color channels
])

### Define class for easy image manipulation

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, imgs_paths, idxs, transform):
        self.imgs_paths = np.array(imgs_paths)[idxs]
        self.transform = transform

    def __len__(self):
        return len(self.imgs_paths)

    def __getitem__(self, idx):
        # Load the image
        img_path = self.imgs_paths[idx]
        img = Image.open(img_path).convert('RGB')
        # Preprocess the image and send it to the chosen device ('cpu' or 'cuda')
        if self.transform:
            img = self.transform(img).to(device)
        return img

In [ ]:
# Get the paths of all image files
train_imgs_paths = sorted(list(Path(train_img_dir).iterdir()))
test_imgs_paths = sorted(list(Path(test_img_dir).iterdir()))

# The DataLoaders contain the ImageDataset class
train_imgs_dataloader = DataLoader(
    ImageDataset(train_imgs_paths, idxs_train, transform), 
    batch_size=batch_size
)
val_imgs_dataloader = DataLoader(
    ImageDataset(train_imgs_paths, idxs_val, transform), 
    batch_size=batch_size
)
test_imgs_dataloader = DataLoader(
    ImageDataset(test_imgs_paths, idxs_test, transform), 
    batch_size=batch_size
)

### Split fMRI data

In [ ]:
lh_fmri_train = lh_fmri[idxs_train]
lh_fmri_val = lh_fmri[idxs_val]
rh_fmri_train = rh_fmri[idxs_train]
rh_fmri_val = rh_fmri[idxs_val]

del lh_fmri, rh_fmri

## Autoencoder

### Get encoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
vae = torch.load(os.path.join('/content/drive/MyDrive/', vae_model_path))
vae.to(device)

train_nodes, _ = get_graph_node_names(vae)
print(train_nodes)

Mounted at /content/drive/
['input', '_kwargs', 'encoder.0.0', 'encoder.0.1', 'encoder.0.2', 'encoder.1.0', 'encoder.1.1', 'encoder.1.2', 'encoder.2.0', 'encoder.2.1', 'encoder.2.2', 'encoder.3.0', 'encoder.3.1', 'encoder.3.2', 'encoder.4.0', 'encoder.4.1', 'encoder.4.2', 'flatten', 'fc_mu', 'fc_var', 'mul', 'exp', 'randn_like', 'mul_1', 'add', 'decoder_input', 'view', 'decoder.0.0', 'decoder.0.1', 'decoder.0.2', 'decoder.1.0', 'decoder.1.1', 'decoder.1.2', 'decoder.2.0', 'decoder.2.1', 'decoder.2.2', 'decoder.3.0', 'decoder.3.1', 'decoder.3.2', 'final_layer.0', 'final_layer.1', 'final_layer.2', 'final_layer.3', 'final_layer.4']


In [ ]:
vae.eval()

VanillaVAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (4): Se

### Construct feature extractor

In [ ]:
encoder_mu = 'fc_mu'

feature_extractor = create_feature_extractor(vae, return_nodes=[encoder_mu])

## Model training

### Extract features

In [ ]:
def extract_features(feature_extractor, dataloader):

    features = []
    for _, d in tqdm(enumerate(dataloader), total=len(dataloader)):
        # downsample data
        downsampled = torch.nn.functional.interpolate(d, size=[64, 64], mode='bilinear')
        # extract features
        ft = feature_extractor(downsampled)
        # Flatten the features
        ft = torch.hstack([torch.flatten(l, start_dim=1) for l in ft.values()])
        # add to features list
        features.append(ft)
        
    return np.vstack(features)

In [ ]:
features_train = extract_features(feature_extractor, train_imgs_dataloader)
features_val = extract_features(feature_extractor, val_imgs_dataloader)
features_test = extract_features(feature_extractor, test_imgs_dataloader)

print('\nTraining images features:')
print(features_train.shape)
print('(Training stimulus images × features)')

print('\nValidation images features:')
print(features_val.shape)
print('(Validation stimulus images × features)')

print('\nTest images features:')
print(features_val.shape)
print('(Test stimulus images × features)')

100%|██████████| 2/2 [04:20<00:00, 130.08s/it]


Training images features:
(41, 1)
(Training stimulus images × features)

Validation images features:
(5, 1)
(Validation stimulus images × features)

Test images features:
(5, 1)
(Test stimulus images × features)


### Define regressor

In [ ]:
class LinearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(MLP, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out

In [ ]:
if (regressor == 'linear'):
    reg_lh = LinearRegression(latent_dim, lh_fmri_train.shape[1])
    reg_rh = LinearRegression(latent_dim, rh_fmri_train.shape[1])
elif (regressor == 'mlp'):
    reg_lh = MLP(latent_dim, lh_fmri_train.shape[1])
    reg_rh = MLP(latent_dim, rh_fmri_train.shape[1])

reg_lh.to(device)
reg_rh.to(device)

print(reg_lh.eval())
print(reg_rh.eval())

LinearRegression(
  (linear): Linear(in_features=100, out_features=18978, bias=True)
)
LinearRegression(
  (linear): Linear(in_features=100, out_features=20220, bias=True)
)


### Train regressor

In [ ]:
def Training(model, features, outputs, epochs=1, batch_size=200):

    # parameters
    lr = 0.005

    # error criterion
    criterion = torch.nn.MSELoss() 

    # optimizer
    optimizer = torch.optim.Adam(model.parameters(),
                               lr=lr,
                               weight_decay=1e-5)
    
    # calculate batches
    batch_iter = int(features.shape[1] / batch_size) + 1
    
    for b in range(batch_iter):

        # get current batch
        x = torch.tensor(features[:, (b * batch_size):(((b+1) * batch_size))])
        y = torch.tensor(outputs[:, (b * batch_size):(((b+1) * batch_size))])

        # add to same device as model
        x.to(device)
        y.to(device)

        for i in range(epochs):

            # predict output
            y_hat = model(x)

            # calculate loss
            loss = criterion(y_hat, y)

            # calculate gradient
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # display progress
            print('\t training loss: %f' % (loss))

In [ ]:
# Train left hemisphere regressor
Training(reg_lh, features_train, lh_fmri_train)

In [ ]:
# Train right hemisphere regressor
Training(reg_rh, features_train, rh_fmri_train)

In [ ]:
# Use fitted linear regressions to predict the validation and test fMRI data
lh_fmri_val_pred = reg_lh(features_val)
lh_fmri_test_pred = reg_lh(features_test)
rh_fmri_val_pred = reg_rh(features_val)
rh_fmri_test_pred = reg_rh(features_test)

AttributeError: ignored

## Evaluate results

### Compute encoding accuracy with Pearson's correlation

In [ ]:
# Empty correlation array of shape: (LH vertices)
lh_correlation = np.zeros(lh_fmri_val_pred.shape[1])
# Correlate each predicted LH vertex with the corresponding ground truth vertex
for v in tqdm(range(lh_fmri_val_pred.shape[1])):
    lh_correlation[v] = corr(lh_fmri_val_pred[:,v], lh_fmri_val[:,v])[0]

# Empty correlation array of shape: (RH vertices)
rh_correlation = np.zeros(rh_fmri_val_pred.shape[1])
# Correlate each predicted RH vertex with the corresponding ground truth vertex
for v in tqdm(range(rh_fmri_val_pred.shape[1])):
    rh_correlation[v] = corr(rh_fmri_val_pred[:,v], rh_fmri_val[:,v])[0]

### Visualize encoding accuracy

In [ ]:
# Load the ROI classes mapping dictionaries
roi_mapping_files = ['mapping_prf-visualrois.npy', 'mapping_floc-bodies.npy',
    'mapping_floc-faces.npy', 'mapping_floc-places.npy',
    'mapping_floc-words.npy', 'mapping_streams.npy']
roi_name_maps = []
for r in roi_mapping_files:
    roi_name_maps.append(np.load(os.path.join(args.data_dir, 'roi_masks', r),
        allow_pickle=True).item())

# Load the ROI brain surface maps
lh_challenge_roi_files = ['lh.prf-visualrois_challenge_space.npy',
    'lh.floc-bodies_challenge_space.npy', 'lh.floc-faces_challenge_space.npy',
    'lh.floc-places_challenge_space.npy', 'lh.floc-words_challenge_space.npy',
    'lh.streams_challenge_space.npy']
rh_challenge_roi_files = ['rh.prf-visualrois_challenge_space.npy',
    'rh.floc-bodies_challenge_space.npy', 'rh.floc-faces_challenge_space.npy',
    'rh.floc-places_challenge_space.npy', 'rh.floc-words_challenge_space.npy',
    'rh.streams_challenge_space.npy']
lh_challenge_rois = []
rh_challenge_rois = []
for r in range(len(lh_challenge_roi_files)):
    lh_challenge_rois.append(np.load(os.path.join(args.data_dir, 'roi_masks',
        lh_challenge_roi_files[r])))
    rh_challenge_rois.append(np.load(os.path.join(args.data_dir, 'roi_masks',
        rh_challenge_roi_files[r])))

# Select the correlation results vertices of each ROI
roi_names = []
lh_roi_correlation = []
rh_roi_correlation = []
for r1 in range(len(lh_challenge_rois)):
    for r2 in roi_name_maps[r1].items():
        if r2[0] != 0: # zeros indicate to vertices falling outside the ROI of interest
            roi_names.append(r2[1])
            lh_roi_idx = np.where(lh_challenge_rois[r1] == r2[0])[0]
            rh_roi_idx = np.where(rh_challenge_rois[r1] == r2[0])[0]
            lh_roi_correlation.append(lh_correlation[lh_roi_idx])
            rh_roi_correlation.append(rh_correlation[rh_roi_idx])
roi_names.append('All vertices')
lh_roi_correlation.append(lh_correlation)
rh_roi_correlation.append(rh_correlation)

# Create the plot
lh_median_roi_correlation = [np.median(lh_roi_correlation[r])
    for r in range(len(lh_roi_correlation))]
rh_median_roi_correlation = [np.median(rh_roi_correlation[r])
    for r in range(len(rh_roi_correlation))]
plt.figure(figsize=(18,6))
x = np.arange(len(roi_names))
width = 0.30
plt.bar(x - width/2, lh_median_roi_correlation, width, label='Left Hemisphere')
plt.bar(x + width/2, rh_median_roi_correlation, width,
    label='Right Hemishpere')
plt.xlim(left=min(x)-.5, right=max(x)+.5)
plt.ylim(bottom=0, top=1)
plt.xlabel('ROIs')
plt.xticks(ticks=x, labels=roi_names, rotation=60)
plt.ylabel('Median Pearson\'s $r$')
plt.legend(frameon=True, loc=1);

### Submit results

In [ ]:
lh_fmri_test_pred = lh_fmri_test_pred.astype(np.float32)
rh_fmri_test_pred = rh_fmri_test_pred.astype(np.float32)

In [ ]:
np.save(os.path.join(args.subject_submission_dir, 'lh_pred_test.npy'), lh_fmri_test_pred)
np.save(os.path.join(args.subject_submission_dir, 'rh_pred_test.npy'), rh_fmri_test_pred)